In [8]:
# https://gist.github.com/iiLaurens/81b1b47f6259485c93ce6f0cdd17490a
clickable = """
window.scrollTo(0, 0)
var bodyRect = document.body.getBoundingClientRect();

var items = Array.prototype.slice.call(
  document.querySelectorAll('*')
).map(function(element) {
  var rect=element.getBoundingClientRect();
  return {
    element: element,
    include: (element.tagName === "BUTTON" || element.tagName === "A" || (element.onclick != null) || window.getComputedStyle(element).cursor == "pointer"),
    rect: {left: Math.max(rect.left - bodyRect.x, 0),
           top: Math.max(rect.top - bodyRect.y, 0),
           right: Math.min(rect.right - bodyRect.x, document.body.clientWidth),
           bottom: Math.min(rect.bottom - bodyRect.y, document.body.clientHeight)},
    text: element.textContent.trim().replace(/\s{2,}/g, ' ')
  };
}).filter(item =>
  item.include && ((item.rect.right - item.rect.left) * (item.rect.bottom - item.rect.top) >= 20));

// Only keep inner clickable items
items = items.filter(x => !items.some(y => x.element.contains(y.element) && !(x == y)))

// Lets create a floating border on top of these elements that will always be visible
items.forEach(function(item) {
  newElement = document.createElement("div");
  newElement.style.outline = "2px dashed rgba(255,0,0,.75)";
  newElement.style.position = "absolute";
  newElement.style.left = item.rect.left + "px";
  newElement.style.top = item.rect.top + "px";
  newElement.style.width = (item.rect.right - item.rect.left) + "px";
  newElement.style.height = (item.rect.bottom - item.rect.top) + "px";
  newElement.style.pointerEvents = "none";
  newElement.style.boxSizering = "border-box";
  newElement.style.zIndex = 2147483647;
  document.body.appendChild(newElement);
  })

"""

In [12]:
writeable = """
window.scrollTo(0, 0);
var bodyRect = document.body.getBoundingClientRect();

var items = Array.prototype.slice.call(document.querySelectorAll('*')).map(function(element) {
  var rect = element.getBoundingClientRect();
  var isInputField = ['INPUT', 'TEXTAREA'].includes(element.tagName);
  var include = (
    element.tagName === 'BUTTON' ||
    element.tagName === 'A' ||
    (element.onclick != null) ||
    window.getComputedStyle(element).cursor == 'pointer' ||
    isInputField
  );

  return {
    element: element,
    include: include,
    rect: {
      left: Math.max(rect.left - bodyRect.x, 0),
      top: Math.max(rect.top - bodyRect.y, 0),
      right: Math.min(rect.right - bodyRect.x, document.body.clientWidth),
      bottom: Math.min(rect.bottom - bodyRect.y, document.body.clientHeight),
    },
    text: element.textContent.trim().replace(/\s{2,}/g, ' '),
  };
}).filter(item => item.include && ((item.rect.right - item.rect.left) * (item.rect.bottom - item.rect.top) >= 20));

// Only keep inner clickable items
items = items.filter(x => !items.some(y => x.element.contains(y.element) && !(x == y)));

// Create a floating border on top of these elements that will always be visible
items.forEach(function(item) {
  var newElement = document.createElement('div');
  newElement.style.outline = '2px dashed rgba(0, 255, 0, .75)'; // Change the color to green
  newElement.style.position = 'absolute';
  newElement.style.left = item.rect.left + 'px';
  newElement.style.top = item.rect.top + 'px';
  newElement.style.width = (item.rect.right - item.rect.left) + 'px';
  newElement.style.height = (item.rect.bottom - item.rect.top) + 'px';
  newElement.style.pointerEvents = 'none';
  newElement.style.boxSizing = 'border-box';
  newElement.style.zIndex = 2147483647;
  document.body.appendChild(newElement);
});


"""

In [1]:
text_region = """
window.scrollTo(0, 0);
var bodyRect = document.body.getBoundingClientRect();

var items = Array.prototype.slice.call(document.querySelectorAll('*')).map(function(element) {
  var rect = element.getBoundingClientRect();
  var isInputField = ['INPUT', 'TEXTAREA'].includes(element.tagName);
  var include = (
    element.tagName === 'BUTTON' ||
    element.tagName === 'A' ||
    (element.onclick != null) ||
    window.getComputedStyle(element).cursor == 'pointer' ||
    isInputField ||
    (element.textContent.trim().length > 0) // Check if the element contains text
  );

  return {
    element: element,
    include: include,
    rect: {
      left: Math.max(rect.left - bodyRect.x, 0),
      top: Math.max(rect.top - bodyRect.y, 0),
      right: Math.min(rect.right - bodyRect.x, document.body.clientWidth),
      bottom: Math.min(rect.bottom - bodyRect.y, document.body.clientHeight),
    },
    text: element.textContent.trim().replace(/\s{2,}/g, ' '),
  };
}).filter(item => item.include && ((item.rect.right - item.rect.left) * (item.rect.bottom - item.rect.top) >= 20));

// Only keep inner clickable items
items = items.filter(x => !items.some(y => x.element.contains(y.element) && !(x == y)));

// Create a floating border on top of these elements that will always be visible
items.forEach(function(item) {
  var newElement = document.createElement('div');
  newElement.style.outline = '2px dashed rgba(0, 0, 0, .75)'; // Change the color to black
  newElement.style.position = 'absolute';
  newElement.style.left = item.rect.left + 'px';
  newElement.style.top = item.rect.top + 'px';
  newElement.style.width = (item.rect.right - item.rect.left) + 'px';
  newElement.style.height = (item.rect.bottom - item.rect.top) + 'px';
  newElement.style.pointerEvents = 'none';
  newElement.style.boxSizing = 'border-box';
  newElement.style.zIndex = 2147483647;
  document.body.appendChild(newElement);
});

"""

In [2]:
def start_browesr():
  from selenium import webdriver
  from selenium.webdriver.chrome.service import Service

  
  chrome_options = webdriver.ChromeOptions()
  #chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--lang=en')
  #chrome_options.headless = True
  return webdriver.Remote("http://host.docker.internal:4444/wd/hub",options=chrome_options)

In [3]:

def web_driver_to_image(wd,prefix=""):
  import re
  import os
  from urllib.parse import urlparse, urlunparse
  
  url = wd.current_url
  # Parse the URL
  parsed_url = urlparse(url)

  # Remove the query parameters
  clean_url = urlunparse((parsed_url.scheme, parsed_url.netloc, parsed_url.path, "", "", ""))
  file_name = re.sub(r'[^a-zA-Z0-9_-]', '_', clean_url)
  file_name = f"{prefix}{file_name}"

  wd.save_screenshot(f"{file_name}.png")
  return file_name

In [17]:
import time
urls = ["https://www.google.com","https://www.bbc.com/"]

for url in urls:
    try:
        wd = start_browesr()
        wd.get(url)
        time.sleep(5)
        wd.execute_script(clickable)
        wd.execute_script(writeable)
        wd.execute_script(text_region)
        web_driver_to_image(wd)
    finally:
        wd.close()
        wd.quit()